In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

from sklearn.metrics import *
from copy import deepcopy
import matplotlib.pyplot as plt
import math


from tqdm.auto import tqdm
import seaborn as sns

from src.helpers import *
from src.visualize import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
from torch.utils.data import Dataset
import pickle

import optuna

from hypnettorch.hnets import HMLP
from hypnettorch.mnets import SimpleRNN


device = torch.device('cpu') #suposed to be cuda
dtype = torch.float32

import sys
sys.path.append("c:\\Users\\nerea\\OneDrive\\Documentos\\EPFL_MASTER\\PDM\\Project\\PyalData")
# to change for the actual path where PyalData has been cloned

/home/nerea/anaconda3/envs/sinthlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = './Data/Processed_Data/Tidy_Sansa_13_04.pkl'

with open(data_path, 'rb') as file:
    df = pickle.load(file)

In [3]:
baseline_df = df.loc[df.type == 'BASELINE'].reset_index()
stim_df = df.loc[df.type == 'TONIC'].reset_index()

In [4]:
X_train_b, y_train_b, X_val_b, y_val_b, X_test_b, y_test_b, info_train_b, info_val_b, info_test_b = train_test_split(baseline_df, train_variable = 'both_rates', 
                                                                                                   target_variable = 'target_pos', num_folds = 5)

Test trials  4
Val trials 3


In [5]:
X_train_s, y_train_s, X_val_s, y_val_s, X_test_s, y_test_s, info_train_s, info_val_s, info_test_s = train_test_split(stim_df, train_variable = 'both_rates', 
                                                                                                   target_variable = 'target_pos', num_folds = 5)

Test trials  8
Val trials 7


In [6]:
# Test one of the folds first
fold_num = 'fold0'
fold = 0

print('We are testing the optimization method on fold ', fold)

def input_mats(x,y, seq_length = 75):
    x = x[fold_num]
    y = y[fold_num]
    x = x.reshape(x.shape[0] // seq_length, seq_length, x.shape[1])  
    y = y.reshape(y.shape[0] // seq_length, seq_length, y.shape[1])  
    return x,y

x_train_base, y_train_base = input_mats(X_train_b, y_train_b)
x_train_stim, y_train_stim = input_mats(X_train_s, y_train_s)

x_val_base, y_val_base = input_mats(X_val_b, y_val_b)
x_val_stim, y_val_stim = input_mats(X_val_s, y_val_s)

x_test_base, y_test_base = input_mats(X_test_b, y_test_b)
x_test_stim, y_test_stim = input_mats(X_test_s, y_test_s)

seed = 42
torch.manual_seed(seed)

We are testing the optimization method on fold  0


In [7]:
# Specify that we want our tensors on the GPU and in float32
device = torch.device('cuda:0') #suposed to be cuda
dtype = torch.float32
path_to_models = './Models'

num_dim_output = y_train_stim.shape[2]
num_features = x_train_stim.shape[2]

seed = 42
torch.manual_seed(seed)

In [8]:
# Define parameters for training
num_epochs = 50
batch_size_train = 25
batch_size_val = 25

In [9]:
class SequenceDataset(Dataset):

    def __init__(self, y, X, seq_length):
        """
        Initializes the SequenceDataset.
        
        Args:
            y (torch.Tensor): The target labels for each sequence.
            X (torch.Tensor): The input sequences.
            sequence_length (int): The desired length of each sequence.
        """
        self.sequence_length = seq_length
        self.y = torch.tensor(y)
        self.X = torch.tensor(X)

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        return self.X.shape[0] * self.X.shape[1]

    def __getitem__(self, i): 
        """
        Gets the i-th sample from the dataset.
        
        Args:
            i (int): Index of the desired sample.
        
        Returns:
            xx (torch.Tensor): Input sequence of length sequence_length.
            yy (torch.Tensor): Corresponding target sequence.
        """
        trial_index = i // self.X.shape[1]
        point_index = i % self.X.shape[1]
        
        if point_index > self.sequence_length - 1:
            point_start = point_index - self.sequence_length
            xx = self.X[trial_index, point_start:point_index, :]
            yy = self.y[trial_index, point_start+1:point_index+1, :]
        else:
            padding_x = self.X[trial_index, 0:1, :].repeat(self.sequence_length - point_index, 1)
            padding_y = self.y[trial_index, 0:1, :].repeat(self.sequence_length - point_index - 1, 1)
            xx = self.X[trial_index, 0:point_index, :]
            xx = torch.cat((padding_x, xx), dim=0)
            yy = self.y[trial_index, 0:point_index + 1, :]
            yy = torch.cat((padding_y, yy), dim=0)
            
        return xx, yy

In [10]:
def huber_loss(X, y, delta):
    residual = torch.abs(X - y)
    condition = residual < delta
    loss = torch.where(condition, 0.5 * residual**2, delta * residual - 0.5 * delta**2)
    return loss.mean()

In [11]:
def reg_hnet_noweights(weights, alpha, l1_ratio):
    
    """
    Implement an L1-L2 penalty on the norm of the model weights.

    model: MLP
    alpha: scaling parameter for the regularization.
    l1_ratio: mixing parameter between L1 and L2 loss.

    Returns:
    reg: regularization term
    """
    l1_loss = 0
    l2_loss = 0

    # Accumulate L1 and L2 losses for weight matrices in the model
    for weight_tensor in weights[1:2]:
        l1_loss += torch.sum(torch.abs(weight_tensor))
        l2_loss += torch.sum(weight_tensor.pow(2))

    reg = l1_ratio * l1_loss + (1 - l1_ratio) * l2_loss

    reg = alpha * reg

    return reg

In [12]:
# Fit the LSTM model
def train_model_optuna(trial):
    
    ###########################################################
    ### Definition of inputs
    ###########################################################
    X_base = x_train_base
    Y_base = y_train_base
    X_val_base = x_val_base
    Y_val_base = y_val_base

    X_stim = x_train_stim
    Y_stim = y_train_stim
    X_val_stim = x_val_stim
    Y_val_stim = y_val_stim


    ###########################################################
    ### Definition of hyperparameters and model for main network
    ###########################################################
    hidden_units = trial.suggest_int('hidden_units', 10, 50)
    input_rec = trial.suggest_int('input_rec', 30, 60)
    seq_length_LSTM = trial.suggest_int('seq_length_LSTM', 10, 20)


    model = SimpleRNN(n_in=num_features, rnn_layers=(hidden_units,), 
                    fc_layers_pre=(input_rec,), fc_layers=(num_dim_output,),
                    use_lstm = True, no_weights = True).to(device)
    

    ###########################################################
    ### Definition of hyperparameters and model for hypernetwork
    ###########################################################
    num_conditions = 2 # fixed, as we only have baseline and stimulation
    size_task_embedding = trial.suggest_int('size_task_embedding', 4, 50)
    num_units_hnet = trial.suggest_int('num_units_hnet', 10,50)
    num_layers_hnet = trial.suggest_int('num_layers_hnet', 1,4)
    layers = [num_units_hnet for i in range(num_layers_hnet)]
    # Model
    hnet = HMLP(model.param_shapes, uncond_in_size=0,
             cond_in_size=size_task_embedding,
            layers=layers, 
            num_cond_embs=num_conditions).to(device)
    
    hnet.apply_hyperfan_init(mnet=model)

    ###########################################################
    ### Fixed training parameters
    ###########################################################
    num_epochs= 1000
    early_stop = 5
    

    ###########################################################
    ### Defining training hyperparameters to optimize
    ###########################################################

    # Set up the optimizer with the specified learning rate
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log = True)
    # Set up the optimizer with the specified learning rate
    optimizer = torch.optim.Adam(hnet.internal_params, lr=lr)

    # Set up a learning rate scheduler
    lr_step_size = 10 # trial.suggest_int('lr_step_size', 5, 15)
    lr_gamma = 0.9 # trial.suggest_float('lr_gamma', 1, 1.5)
    scheduler = lr_scheduler.StepLR(optimizer, 
                                    step_size=lr_step_size, 
                                    gamma=lr_gamma)
    # hyperparameter for huber loss
    delta = trial.suggest_int('huber_delta', 5, 10) #was 8 before

    # hyperparameter for regularizer
    alpha = trial.suggest_float('alpha_reg', 1e-7, 1e-3, log = True) # was 1e-5 before
    l1_ratio =trial.suggest_float('l1_ratio_reg', 0.2, 0.8) #was 0.5 before

    # Reshaping data and creating batchs
    batch_size_train = trial.suggest_int('batch_size_train', 20, 50)
    batch_size_val = trial.suggest_int('batch_size_val',20, 50)

    train_dataset_baseline = SequenceDataset(
    y_train_base,    x_train_base,    seq_length=seq_length_LSTM)
    train_dataset_stim = SequenceDataset(
    y_train_stim,    x_train_stim,    seq_length=seq_length_LSTM)
    val_dataset_baseline = SequenceDataset(
    y_val_base,    x_val_base,    seq_length=seq_length_LSTM)
    val_dataset_stim = SequenceDataset(
    y_val_stim,    x_val_stim,    seq_length=seq_length_LSTM)

    loader_train_b = data.DataLoader(train_dataset_baseline, batch_size=batch_size_train, shuffle=True)
    loader_train_s = data.DataLoader(train_dataset_stim, batch_size=batch_size_train, shuffle=True)

    loader_val_b = data.DataLoader(val_dataset_baseline, batch_size=batch_size_val, shuffle=True)
    loader_val_s = data.DataLoader(val_dataset_stim, batch_size=batch_size_val, shuffle=True)


    #####################################################
    ########## START TRAINING 
    #####################################################

    # Keep track of the best model's parameters and loss
    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1e8

    # Enable anomaly detection for debugging
    torch.autograd.set_detect_anomaly(True)

    # Track the train and validation loss
    train_losses = []
    val_losses = []
    # Counters for early stopping
    not_increased = 0
    end_train = 0
    
    regularizer = True

    # Loop through epochs
    for epoch in np.arange(num_epochs):
        for phase in ['train', 'val']:
            # set model to train/validation as appropriate
            if phase == 'train':
                model.train()
                loaders = zip(loader_train_b, loader_train_s)
            else:
                model.eval()
                loaders = zip(loader_val_b, loader_val_s)

            # Initialize variables to track loss and batch size
            running_loss = 0
            running_size = 0        

            # Iterate over batches in the loader
            for data_b, data_s in loaders:

                # Define data for this batch
                x_b = data_b[0].to('cuda')
                y_b = data_b[1].to('cuda')
                x_s = data_s[0].to('cuda')
                y_s = data_s[1].to('cuda')

                
                if phase == "train":
                    with torch.set_grad_enabled(True):
                        optimizer.zero_grad()

                        # Compute BASELINE loss.
                        W_base = hnet(cond_id=0)
                        base_P = model.forward(x_b, weights=W_base)
                        base_P = torch.squeeze(base_P) # torch.sigmoid(base_P))
                        loss_base = huber_loss(base_P, y_b, delta = delta)
                        
                        
                        # Compute STIMULATION loss.
                        W_stim = hnet(cond_id=1)
                        stim_P = model.forward(x_s, weights=W_stim)
                        stim_P = torch.squeeze(stim_P) #torch.sigmoid(stim_P))
                        loss_stim = huber_loss(stim_P, y_s, delta = delta)

                        # Combine loss for 2 tasks
                        loss_t = loss_base + loss_stim    #only for printing

                        # Add regularization to the loss in the training phase
                        if regularizer is not None:
                            loss_stim_reg = loss_stim + reg_hnet_noweights(W_stim, l1_ratio, alpha)
                            loss_base_reg = loss_base + reg_hnet_noweights(W_base, l1_ratio, alpha)
                            # Combine loss for 2 tasks
                            loss_t_r = loss_base_reg + loss_stim_reg

                        else:               
                            loss_t_r = loss_t 

                        # Compute gradients and perform an optimization step
                        loss_t_r.backward()
                        optimizer.step()
                else:
                    # just compute the loss in validation phase
                    W_base = hnet(cond_id=0)
                    base_P = model.forward(x_b, weights=W_base)
                    base_P = torch.squeeze(base_P) #torch.sigmoid(base_P))
                    loss_base = huber_loss(base_P, y_b, delta = delta)

                    W_stim = hnet(cond_id=1)
                    stim_P = model.forward(x_s, weights=W_stim)
                    stim_P = torch.squeeze(stim_P) #torch.sigmoid(stim_P))
                    loss_stim = huber_loss(stim_P, y_s, delta = delta)

                    loss_t = loss_base + loss_stim

                # Ensure the loss is finite
                assert torch.isfinite(loss_t)
                assert torch.isfinite(loss_t_r)
                running_loss += loss_t.item()
                running_size += 1

            # compute the train/validation loss and update the best
            # model parameters if this is the lowest validation loss yet
            running_loss /= running_size
            if phase == "train":
                train_losses.append(running_loss)
            else:
                val_losses.append(running_loss)
                
                # Update best model parameters if validation loss improves
                if running_loss < best_loss:
                    best_loss = running_loss
                    best_model_wts = deepcopy(model.state_dict())
                    not_increased = 0
                else:
                    # Perform early stopping if validation loss doesn't improve
                    if epoch > 10:
                        not_increased += 1
                        # print('Not increased : {}/5'.format(not_increased))
                        if not_increased == early_stop:
                            print('Decrease LR')
                            for g in optimizer.param_groups:
                                g['lr'] = g['lr'] / 2
                            not_increased = 0
                            end_train += 1
                        
                        if end_train == 2:
                            model.load_state_dict(best_model_wts)
                            return best_loss
                        

        # Update learning rate with the scheduler
        scheduler.step()
        #print("Epoch {:03} Train {:.4f} Val {:.4f}".format(epoch, train_losses[-1], val_losses[-1]))


        trial.report(val_losses[-1], epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    

    return best_loss


In [13]:
study = optuna.create_study(direction="minimize")

# Set pruning options
study.pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5)

study.optimize(train_model_optuna, n_trials=30)

importance_scores = optuna.importance.get_param_importances(study)

# Print importance scores
for param, score in importance_scores.items():
    print(f"{param}: {score}")

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print('Study statistics: ')
print("Number of finished trials: ", len(study.trials))
print("Number of pruned trials: ", len(pruned_trials))
print("Number of complete trials: ", len(complete_trials))

print('Best trial: ')
trial = study.best_trial

print("Loss: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-04-22 16:49:57,077] A new study created in memory with name: no-name-e83fc5d1-77d2-4094-bc5a-4993a98882e8


Creating a simple RNN with 10428 weights.
Created MLP Hypernet.
Hypernetwork with 357529 weights and 10428 outputs (compression ratio: 34.29).
The network consists of 357489 unconditional weights (357489 internally maintained) and 40 conditional weights (40 internally maintained).
Epoch 000 Train 4.7249 Val 3.7838
Epoch 001 Train 3.1366 Val 2.4792
Epoch 002 Train 1.8752 Val 1.4785
Epoch 003 Train 1.0148 Val 0.8099
Epoch 004 Train 0.5244 Val 0.4099
Epoch 005 Train 0.2646 Val 0.2443
Epoch 006 Train 0.1701 Val 0.1808
Epoch 007 Train 0.1355 Val 0.1501
Epoch 008 Train 0.1131 Val 0.1369
Epoch 009 Train 0.1040 Val 0.1244
Epoch 010 Train 0.0988 Val 0.1249
Epoch 011 Train 0.0921 Val 0.1155
Epoch 012 Train 0.0884 Val 0.1111
Epoch 013 Train 0.0832 Val 0.1082
Epoch 014 Train 0.0789 Val 0.1050
Epoch 015 Train 0.0771 Val 0.1033
Epoch 016 Train 0.0730 Val 0.0978
Epoch 017 Train 0.0732 Val 0.1011
Epoch 018 Train 0.0692 Val 0.1015
Epoch 019 Train 0.0670 Val 0.0959
Epoch 020 Train 0.0656 Val 0.0880
Epoc

[I 2024-04-22 17:15:29,646] Trial 0 finished with value: 0.05891955678222606 and parameters: {'hidden_units': 19, 'input_rec': 43, 'seq_length_LSTM': 11, 'size_task_embedding': 20, 'num_units_hnet': 33, 'num_layers_hnet': 3, 'lr': 2.0059438162661392e-05, 'huber_delta': 9, 'alpha_reg': 0.00011252632910055825, 'l1_ratio_reg': 0.3651107054885052, 'batch_size_train': 32, 'batch_size_val': 36}. Best is trial 0 with value: 0.05891955678222606.


Decrease LR
Creating a simple RNN with 8669 weights.
Created MLP Hypernet.
Hypernetwork with 418560 weights and 8669 outputs (compression ratio: 48.28).
The network consists of 418462 unconditional weights (418462 internally maintained) and 98 conditional weights (98 internally maintained).
Epoch 000 Train 1.5703 Val 1.4964
Epoch 001 Train 1.1810 Val 1.1705
Epoch 002 Train 0.8924 Val 0.9223
Epoch 003 Train 0.6403 Val 0.6737
Epoch 004 Train 0.4489 Val 0.5003
Epoch 005 Train 0.3220 Val 0.3741
Epoch 006 Train 0.2440 Val 0.2894
Epoch 007 Train 0.1925 Val 0.2560
Epoch 008 Train 0.1551 Val 0.2171
Epoch 009 Train 0.1329 Val 0.2003
Epoch 010 Train 0.1153 Val 0.1839
Epoch 011 Train 0.1028 Val 0.1708
Epoch 012 Train 0.0914 Val 0.1611
Epoch 013 Train 0.0830 Val 0.1510
Epoch 014 Train 0.0764 Val 0.1391
Epoch 015 Train 0.0711 Val 0.1302
Epoch 016 Train 0.0649 Val 0.1194
Epoch 017 Train 0.0614 Val 0.1257
Epoch 018 Train 0.0583 Val 0.1151
Epoch 019 Train 0.0542 Val 0.1199
Epoch 020 Train 0.0514 Val 0

[I 2024-04-22 17:31:01,764] Trial 1 finished with value: 0.07427431724078508 and parameters: {'hidden_units': 14, 'input_rec': 42, 'seq_length_LSTM': 14, 'size_task_embedding': 49, 'num_units_hnet': 47, 'num_layers_hnet': 1, 'lr': 1.3070208278080213e-05, 'huber_delta': 9, 'alpha_reg': 0.0002070864599485419, 'l1_ratio_reg': 0.3388216255543003, 'batch_size_train': 38, 'batch_size_val': 23}. Best is trial 0 with value: 0.05891955678222606.


...

Decrease LR
Creating a simple RNN with 13260 weights.
Created MLP Hypernet.
Hypernetwork with 212704 weights and 13260 outputs (compression ratio: 16.04).
The network consists of 212670 unconditional weights (212670 internally maintained) and 34 conditional weights (34 internally maintained).
Epoch 000 Train 0.1582 Val 0.0839
Epoch 001 Train 0.0383 Val 0.0497
Epoch 002 Train 0.0240 Val 0.0449
Epoch 003 Train 0.0179 Val 0.0445
Epoch 004 Train 0.0143 Val 0.0428
Epoch 005 Train 0.0108 Val 0.0476
Epoch 006 Train 0.0083 Val 0.0441
Epoch 007 Train 0.0063 Val 0.0429
Epoch 008 Train 0.0051 Val 0.0436
Epoch 009 Train 0.0042 Val 0.0404
Epoch 010 Train 0.0035 Val 0.0421
Epoch 011 Train 0.0030 Val 0.0431
Epoch 012 Train 0.0026 Val 0.0387
Epoch 013 Train 0.0022 Val 0.0416
Epoch 014 Train 0.0019 Val 0.0412
Epoch 015 Train 0.0016 Val 0.0425
Epoch 016 Train 0.0015 Val 0.0420
Decrease LR
Epoch 017 Train 0.0013 Val 0.0430
Epoch 018 Train 0.0012 Val 0.0388
Epoch 019 Train 0.0011 Val 0.0415
Epoch 020 Trai

[I 2024-04-22 19:45:03,282] Trial 28 finished with value: 0.03867565437342744 and parameters: {'hidden_units': 31, 'input_rec': 36, 'seq_length_LSTM': 18, 'size_task_embedding': 17, 'num_units_hnet': 15, 'num_layers_hnet': 2, 'lr': 0.005297718408174701, 'huber_delta': 5, 'alpha_reg': 2.176486136462753e-05, 'l1_ratio_reg': 0.20571143159122676, 'batch_size_train': 41, 'batch_size_val': 43}. Best is trial 13 with value: 0.034108375129166556.


Decrease LR
Creating a simple RNN with 9578 weights.
Created MLP Hypernet.
Hypernetwork with 221808 weights and 9578 outputs (compression ratio: 23.16).
The network consists of 221768 unconditional weights (221768 internally maintained) and 40 conditional weights (40 internally maintained).
Epoch 000 Train 0.0903 Val 0.0756
Epoch 001 Train 0.0337 Val 0.0550
Epoch 002 Train 0.0216 Val 0.0481
Epoch 003 Train 0.0179 Val 0.0454
Epoch 004 Train 0.0157 Val 0.0464
Epoch 005 Train 0.0140 Val 0.0448
Epoch 006 Train 0.0131 Val 0.0436


[I 2024-04-22 19:47:31,841] Trial 29 pruned. 


Epoch 007 Train 0.0121 Val 0.0439
lr: 0.21014374889128268
huber_delta: 0.13013276935793192
batch_size_val: 0.12130770741551816
num_units_hnet: 0.10569991913357972
size_task_embedding: 0.10488262950363018
batch_size_train: 0.07627180230148475
seq_length_LSTM: 0.0706055119627895
input_rec: 0.043272813606277515
l1_ratio_reg: 0.042129665231524185
alpha_reg: 0.03388434194171077
num_layers_hnet: 0.031113185708871262
hidden_units: 0.03055590494539925
Study statistics: 
Number of finished trials:  30
Number of pruned trials:  9
Number of complete trials:  21
Best trial: 
Loss: 0.034108375129166556
Best hyperparameters: {'hidden_units': 26, 'input_rec': 30, 'seq_length_LSTM': 20, 'size_task_embedding': 13, 'num_units_hnet': 11, 'num_layers_hnet': 3, 'lr': 0.008724683475679063, 'huber_delta': 6, 'alpha_reg': 3.3324906702023635e-06, 'l1_ratio_reg': 0.2132256325697216, 'batch_size_train': 50, 'batch_size_val': 50}


In [14]:
import joblib
import json
# Retrieve the best trial
best_trial = study.best_trial

# Get the best hyperparameters
best_params = best_trial.params

# Save the best parameters
with open('stim_0411_best_params.json', 'w') as f:
    json.dump(best_params, f)

### Plotting the study

In [18]:
import plotly

In [19]:
optuna.visualization.plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [20]:
#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [21]:
# Plotting the optimization history of the study.

optuna.visualization.plot_optimization_history(study)

#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

# Plotting the accuracy surface for the hyperparameters involved in the random forest model.

optuna.visualization.plot_contour(study, params=["seq_length_LSTM", "n_hidden_units"]) 

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.